In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
from random import sample 

In [2]:
dataset = 'imagenet'
imdb_name = 'imagenet_train'
imdbval_name = 'imagenet_val'
split = 'train'#train val val1 val2 test
name = 'imagenet_{}'.format(split)
devkit_path = 'data/imagenet/ILSVRC/devkit'
data_path = 'data/imagenet/ILSVRC'

In [3]:
import scipy.io as sio
synsets_image = sio.loadmat(os.path.join(devkit_path,'data','meta_det.mat'))
synsets_video = sio.loadmat(os.path.join(devkit_path,'data','meta_vid.mat'))
syn_I = synsets_image['synsets']
syn_V = synsets_video['synsets']
vid_wnids = [syn_V[0,i][1] for i in range(30)]

In [4]:
vtmp_index = [] # label as 1.
vtmp_index0 = [] # label as 0.
vtmp_index_1 = [] # label as -1.
vtmp_used = []
vtmp_used_len_all = []
vtmp_used_len_sel = []
for i in range(1,201):
    wnid = syn_I[0,i-1][1]
    if wnid in vid_wnids:
        image_set_file = os.path.join(data_path,'ImageSets','DET','train_'+str(i)+'.txt')
        print(image_set_file,os.path.exists(image_set_file))
        with open(image_set_file,'r') as f:
            tmp_index = [x for x in f.readlines()]
            vtmp = []
            for line in tmp_index:
                line = line.strip()
                line = line.split(' ')
                if line[1] == '1':
                    vtmp_index.append(data_path+'/Data/DET/train/'+line[0])
                    vtmp.append(line[0])
                elif line[1] == '0':
                    vtmp_index0.append(data_path+'/Data/DET/train/'+line[0])
                elif line[1] == '-1':
                    vtmp_index_1.append(data_path+'/Data/DET/train/'+line[0])
                else:
                    raise Exception('Error.')
            vtmp_used_len_all.append(len(vtmp))
            if len(vtmp)>2500:
                vtmp = sample(vtmp,k=2500)
            vtmp_used.extend(vtmp)
            vtmp_used_len_sel.append(len(vtmp))
            
print(len(vtmp_index),len(vtmp_index0),len(vtmp_index_1),'sum:',len(vtmp_index)+len(vtmp_index0)+len(vtmp_index_1))
vtmp_used_len_all = np.array(vtmp_used_len_all,dtype = np.float)
vtmp_used_len_sel = np.array(vtmp_used_len_sel,dtype = np.float)
print('vtmp_used_len_all:',vtmp_used_len_all)
print('vtmp_used_len_sel:',vtmp_used_len_sel)

data/imagenet/ILSVRC/ImageSets/DET/train_2.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_4.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_20.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_24.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_26.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_33.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_37.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_39.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_58.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_59.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_64.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_70.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_74.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_84.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_92.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_103.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_105.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_113.txt True
data/imagenet/ILSVRC/ImageSets/DET/train_114.

In [21]:
def cal_det_number(det_counter, max_det_per_category = 2000):
    return np.minimum(det_counter, max_det_per_category)
print(cal_det_number(vtmp_used_len_all))
print(np.sum(cal_det_number(vtmp_used_len_all)))

[1569. 1948. 2000. 1526. 2000. 2000. 2000.  948. 2000. 2000. 1579. 2000.
  838.  786. 1555.  840. 2000. 2000. 1989. 2000.  973. 1389. 2000.  910.
  882. 1216. 2000. 2000. 1266.  834.]
47048.0


In [38]:
with open(os.path.join(data_path,'ImageSets','my_train_VID.txt'),'w') as f:
    for item in vtmp_used:
        f.write(item+' 1\n')

### compare MANet train Det and my train Det

In [39]:
path_MAnet_det_txt = '../MANet_for_Video_Object_Detection/data/ILSVRC2015/ImageSets/DET_train_30classes.txt'
path_my_det_txt = os.path.join(data_path,'ImageSets','my_train_VID.txt')
assert os.path.exists(path_MAnet_det_txt)
assert os.path.exists(path_my_det_txt)

In [40]:
MA_items = []
my_items =[]
with open(path_MAnet_det_txt) as fMA:
    MA_items = [x.split(' ')[0] for x in fMA.readlines()]

with open(path_my_det_txt) as f:
    my_items = ['train/'+x.split(' ')[0] for x in f.readlines()]

MA_items = set(MA_items)
my_items = set(my_items)
print(len(MA_items))
print(len(my_items))

53237
52832


In [41]:
print(len(MA_items-my_items))
print(len(my_items-MA_items))

15889
15484
